In [ ]:
Dataset URL: Corona Virus NLP Tweets data set
https://www.kaggle.com/datasets/datatattle/covid-19-nlp-text-classification


In [23]:
import pandas as pd
train_data = pd.read_csv('Corona_NLP_train.csv',  encoding='latin-1')
train_data.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive
3,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,Positive
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative


In [24]:
train_data = train_data.drop(['UserName', 'ScreenName', 'Location', 'TweetAt'], axis = 1)
train_data.head()

,OriginalTweet,Sentiment
0,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,advice Talk to your neighbours family to excha...,Positive
2,Coronavirus Australia: Woolworths to give elde...,Positive
3,My food stock is not the only one which is emp...,Positive
4,"Me, ready to go at supermarket during the #COV...",Extremely Negative


In [25]:
#train_data = train_data.reset_index(drop = True)
#train_data.head()

In [26]:
# Removing stop words, punctuation and converting to lowercase

import nltk
import string
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stopwordsandpunct = stop_words + list(string.punctuation)

train_data['OriginalTweet'] = train_data['OriginalTweet'].apply(lambda w:' '.join(w for w in w.split() if w not in stopwordsandpunct))
train_data['OriginalTweet'] = train_data['OriginalTweet'].str.lower()
train_data['OriginalTweet']



0        @menyrbie @phil_gahan @chrisitv https://t.co/i...
1        advice talk neighbours family exchange phone n...
2        coronavirus australia: woolworths give elderly...
3        my food stock one empty... please, panic, ther...
4        me, ready go supermarket #covid19 outbreak. no...
                               ...                        
41152    airline pilots offering stock supermarket shel...
41153    response complaint provided citing covid-19 re...
41154    you know itâs getting tough @kameronwilds rat...
41155    is wrong smell hand sanitizer starting turn on...
41156    @tartiicat well new/used rift s going $700.00 ...
Name: OriginalTweet, Length: 41157, dtype: object

In [27]:
# Removing url
import re
def remove_url(tweet):
    tweet = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', tweet)
    return tweet

train_data['OriginalTweet'] = train_data['OriginalTweet'].apply(remove_url)
train_data['OriginalTweet'].head()

0                   @menyrbie @phil_gahan @chrisitv   
1    advice talk neighbours family exchange phone n...
2    coronavirus australia: woolworths give elderly...
3    my food stock one empty... please, panic, ther...
4    me, ready go supermarket #covid19 outbreak. no...
Name: OriginalTweet, dtype: object

In [28]:
# remove mentions and hashtags
def remove_mentions_hashs(tweet):
    tweet = re.sub("@[A-Za-z0-9_]+","", tweet) #Remove mentions
    tweet = re.sub("#[A-Za-z0-9_]+","", tweet) #Remove hashtags
    return tweet

train_data['OriginalTweet'] = train_data['OriginalTweet'].apply(remove_mentions_hashs)
train_data['OriginalTweet'].head()

0                                                     
1    advice talk neighbours family exchange phone n...
2    coronavirus australia: woolworths give elderly...
3    my food stock one empty... please, panic, ther...
4    me, ready go supermarket  outbreak. not i'm pa...
Name: OriginalTweet, dtype: object

In [29]:
import string
def remove_empty_strings1(tweet):
    tweet = re.sub(r"^\s+|\s+$", 'NaN', tweet)
    return tweet
train_data['OriginalTweet'] =train_data['OriginalTweet'].apply(remove_empty_strings1)

In [30]:
import unicodedata
def remove_nonascii(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')# apply compatibility decomposition
    return text
train_data['OriginalTweet'] = train_data['OriginalTweet'].apply(remove_nonascii)
train_data.head()

,OriginalTweet,Sentiment
0,NaN,Neutral
1,advice talk neighbours family exchange phone n...,Positive
2,coronavirus australia: woolworths give elderly...,Positive
3,"my food stock one empty... please, panic, ther...",Positive
4,"me, ready go supermarket outbreak. not i'm pa...",Extremely Negative


In [31]:
# Removing emojis from tweets
# https://stackoverflow.com/a/49146722/330558
import re
def remove_emojis(tweet):
    pat =      re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return pat.sub(r'', tweet)

train_data['OriginalTweet'] =train_data['OriginalTweet'].apply(remove_emojis)
train_data.head()

,OriginalTweet,Sentiment
0,NaN,Neutral
1,advice talk neighbours family exchange phone n...,Positive
2,coronavirus australia: woolworths give elderly...,Positive
3,"my food stock one empty... please, panic, ther...",Positive
4,"me, ready go supermarket outbreak. not i'm pa...",Extremely Negative


In [32]:
## Removing all rows containing NaNs after removing hashtags, URLs
train_data = train_data[train_data['OriginalTweet'] != 'NaN']
train_data.head()

,OriginalTweet,Sentiment
1,advice talk neighbours family exchange phone n...,Positive
2,coronavirus australia: woolworths give elderly...,Positive
3,"my food stock one empty... please, panic, ther...",Positive
4,"me, ready go supermarket outbreak. not i'm pa...",Extremely Negative
5,as news regionas first confirmed covid-19 case...,Positive


In [33]:
# Extracting only the tweets that have sentiment labels - positive and negative 
pos_neg_tweets = train_data.loc[(train_data['Sentiment']=='Negative')|(train_data['Sentiment']=='Positive')]
pos_neg_tweets.reset_index(drop = True)
pos_neg_tweets.head()

,OriginalTweet,Sentiment
1,advice talk neighbours family exchange phone n...,Positive
2,coronavirus australia: woolworths give elderly...,Positive
3,"my food stock one empty... please, panic, ther...",Positive
5,as news regionas first confirmed covid-19 case...,Positive
6,cashier grocery store sharing insights to pro...,Positive


In [34]:
# Now we will perform tokenization
import warnings
warnings.filterwarnings("ignore")

import nltk
from nltk import word_tokenize
tokenizer = nltk.tokenize.WhitespaceTokenizer()
def tokenize(text):
    return tokenizer.tokenize(text)

pos_neg_tweets['OriginalTweet'] = pos_neg_tweets['OriginalTweet'].apply(tokenize)
pos_neg_tweets.head()

,OriginalTweet,Sentiment
1,"[advice, talk, neighbours, family, exchange, p...",Positive
2,"[coronavirus, australia:, woolworths, give, el...",Positive
3,"[my, food, stock, one, empty..., please,, pani...",Positive
5,"[as, news, regionas, first, confirmed, covid-1...",Positive
6,"[cashier, grocery, store, sharing, insights, t...",Positive


In [37]:
import warnings
warnings.filterwarnings("ignore")
# converting token lists to strings using join() on each row or the OriginalTweet column
pos_neg_tweets['OriginalTweet'] = pos_neg_tweets.loc[:, 'OriginalTweet'].apply(lambda x: ', '.join(x))
pos_neg_tweets.head()

,OriginalTweet,Sentiment
1,"advice, talk, neighbours, family, exchange, ph...",Positive
2,"coronavirus, australia:, woolworths, give, eld...",Positive
3,"my, food, stock, one, empty..., please,, panic...",Positive
5,"as, news, regionas, first, confirmed, covid-19...",Positive
6,"cashier, grocery, store, sharing, insights, to...",Positive


In [35]:
pos_neg_tweets.shape

(21336, 2)

In [38]:
# Parsing OriginalTweet column with SpaCy's pipeline
import warnings
warnings.filterwarnings("ignore")

import spacy
nlp = spacy.load('en_core_web_sm')
def parsetweets(text):
    text = text
    parsedtext = nlp(text)
    return parsedtext


pos_neg_tweets['parsed'] = pos_neg_tweets.loc[:,'OriginalTweet'].apply(parsetweets)
pos_neg_tweets.head()

,OriginalTweet,Sentiment,parsed
1,"advice, talk, neighbours, family, exchange, ph...",Positive,"(advice, ,, talk, ,, neighbours, ,, family, ,,..."
2,"coronavirus, australia:, woolworths, give, eld...",Positive,"(coronavirus, ,, australia, :, ,, woolworths, ..."
3,"my, food, stock, one, empty..., please,, panic...",Positive,"(my, ,, food, ,, stock, ,, one, ,, empty, ...,..."
5,"as, news, regionas, first, confirmed, covid-19...",Positive,"(as, ,, news, ,, regionas, ,, first, ,, confir..."
6,"cashier, grocery, store, sharing, insights, to...",Positive,"(cashier, ,, grocery, ,, store, ,, sharing, ,,..."


In [60]:
import scattertext as st
from IPython.display import IFrame, display, HTML
from scattertext import CorpusFromPandas, produce_scattertext_explorer

corpus = st.CorpusFromParsedDocuments(pos_neg_tweets.iloc[:1000,:], category_col ='Sentiment', parsed_col='parsed').build()
html = st.produce_scattertext_explorer(corpus, category = 'Negative', category_name='Negative',
                                      not_category_name='Positive', minimum_term_frequency=10,
                                      width_in_pixels=1000, transform = st.Scalers.log_scale_standardize)
file_name = 'scattertextdemo_plot.html'
open(file_name, 'wb').write(html.encode('utf-8'))
IFrame(src=file_name, width = 800, height = 500)